<a href="https://colab.research.google.com/github/utkarshminhas/violence-detection-fyp/blob/main/notebooks/working_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from glob import glob
from sklearn.utils import shuffle
import numpy as np

In [2]:
processed_dir = '/content/drive/Shareddrives/Datasets/Raw RWF-2000 (without_mog)'
print(len(os.listdir(processed_dir)))
print(len(os.listdir(processed_dir+"/Fight")))
print(len(os.listdir(processed_dir+'/NonFight')))

# image_size = (299, 299)
image_size = (128, 128)
nonFight, fight = (np.array(0), np.array(1))
# nonFight, fight = ((None,0), (None,1))


2
1000
1000


In [3]:
def load_initial_data():
    x = []
    y = []

    # Fights
    allFights = sorted(glob(os.path.join(processed_dir, 'Fight/*')))
    x.extend(allFights)
    y.extend([fight] * len(allFights))

    # Non Fights
    allNonFights = sorted(glob(os.path.join(processed_dir, 'NonFight/*')))
    x.extend(allNonFights)
    y.extend([nonFight] * len(allNonFights))

    return x, y

In [4]:
videoPaths, labels = load_initial_data()
df = pd.DataFrame(list(zip(videoPaths, labels)), 
               columns =['Name', 'Label']) 
df=shuffle(df)
# df

In [5]:
allvideos=df.to_numpy()
videos,labels=[],[]
for video in allvideos:
  videos.append(video[0])
  labels.append(video[1])


# train_videoPaths,train_labels=videos[:1500],labels[:1500]
# val_videoPaths,val_labels=videos[1500:1900],labels[1500:1900]

In [6]:
def get_image(imagePath):
    img = cv2.imread(imagePath, cv2.IMREAD_COLOR) # Read in RGB  directly
    img = cv2.resize(img, image_size)
    img = img / 255.0
    img = img.astype(np.float32)
    return img

In [7]:
counter=0
def preprocess(videoPath, label):
    def myFunction(videoPath):
        global counter
        videoPath = videoPath.decode()
        frames = []
        for frame_path in glob(os.path.join(videoPath, '*')):
            image = get_image(frame_path)
            frames.append(image)
        counter+=1
        print(" returning video number ", counter)
        return np.asarray(frames)
    video = tf.numpy_function(myFunction, [videoPath], [tf.float32])[0]
    # label=tf.convert_to_tensor(label)
    # print(video.shape)
    # label=tf.one_hot(label[0],2)
    return video, label

In [8]:
def tf_dataset(videoPaths, labels, batch_size=5):
    dataset = tf.data.Dataset.from_tensor_slices((videoPaths, labels))
    dataset = dataset.map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [9]:
ds=tf_dataset(videos[:100], labels[:100], batch_size=1)
steps=len(ds)
print(steps)

100


In [ ]:
train_dataset = tf_dataset(train_videoPaths, train_labels, batch_size=1)
val_dataset = tf_dataset(val_videoPaths, val_labels, batch_size=1)

In [ ]:
print(len(train_dataset))
print(len(val_dataset))

In [ ]:
for x, y in train_dataset:
    x = x.numpy()
    y = y.numpy()
    print("One Batch is:", x.shape, y.shape)
    x = x[0] 
    # y = y[0]
    print("One video is:", x.shape, y.shape, end="\n\n")
    # x = x[0] 
    print("One image is:", x.shape, y.shape, end="\n\n")
    x *= 255
    print(y)
    cv2_imshow(x)
    break

In [ ]:
import tensorflow_datasets as tfds
import datetime
ds_numpy = tfds.as_numpy(ds)
iterator = iter(ds_numpy)

In [ ]:
a = datetime.datetime.now()
curr=a
for x,y in iterator:
  print(x.shape,y.shape)
  b = datetime.datetime.now()
  c = b - a
  a=b
  print(c.seconds)
c=datetime.datetime.now()
time_taken=c-curr
print(time_taken.seconds)

In [ ]:
# print(iterator.shape)
x=next(iterator)# return a tuple of (numpy.ndarray,numpy.ndarray)
print(type(x[0]),type(x[1]))

print()
print(x[0].shape,x[1].shape)#(2, 30, 128, 128, 3) (2,)
# for ex in ds_numpy:
#   # `{'image': np.array(shape=(28, 28, 1)), 'labels': np.array(shape=())}`
#   print(ex[0][0].shape)

In [10]:
from tensorflow.keras import layers
import numpy as np
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, \
Flatten, LSTM, Dropout, BatchNormalization
from keras import models
import itertools

import keras
from keras import optimizers
# import tensorflow_addons as tfa

import datetime

In [11]:
input_shape = (30, image_size[0], image_size[1], 3)
num_epochs = 5
print(input_shape)

(30, 128, 128, 3)


In [12]:
strategy= tf.distribute.MultiWorkerMirroredStrategy()

with strategy.scope():
    model = models.Sequential(
        [
            TimeDistributed(
                Conv2D(32, (3, 3), activation=tf.nn.relu), 
                input_shape=input_shape
            ),
            TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))),
            TimeDistributed(Conv2D(32, (4, 4), activation=tf.nn.relu)),
            TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
            TimeDistributed(Conv2D(32, (4, 4), activation=tf.nn.relu)),
            TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
            TimeDistributed(Flatten()),
            # Dropout(0.6),
            LSTM(256, return_sequences=False, dropout=0.5,),
            Dropout(0.7),
            Flatten(),
            Dense(64, activation=tf.nn.relu),
            Dense(32, activation=tf.nn.relu),
            Dense(1, activation=tf.nn.sigmoid)
        ]
    )

    model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 126, 126, 32)  896       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 125, 125, 32)  0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 122, 122, 32)  16416     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 61, 61, 32)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 58, 58, 32)    16416     
___________________________________

In [13]:
optimizer = optimizers.Adam()

model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=['accuracy']
)

def gen():
  print()

In [ ]:
autotune=

In [14]:

with tf.device('/device:GPU:0'):
    history = model.fit(
        ds,
        # iterator,
        # x=np.array(next(iterator)[0]),
        # y=np.array(next(iterator)[1]),
        # validation_data=val_dataset,
        epochs=num_epochs,
        steps_per_epoch=100,
        # validation_steps=200,
        batch_size=1,
        # validation_batch_size=2,
        verbose=2,
        use_multiprocessing=True,
        workers=16
        
        
    )

Epoch 1/5
 returning video number  1
 returning video number  2
 returning video number  3
 returning video number  4
 returning video number  5
 returning video number  6
 returning video number  7
 returning video number  8
 returning video number  9
 returning video number  10
 returning video number  11
 returning video number  12
 returning video number  13
 returning video number  14
 returning video number  15
 returning video number  16
 returning video number  17
 returning video number  18
 returning video number  19
 returning video number  20
 returning video number  21
 returning video number  22
 returning video number  23
 returning video number  24
 returning video number  25
 returning video number  26
 returning video number  27
 returning video number  28
 returning video number  29
 returning video number  30
 returning video number  31
 returning video number  32
 returning video number  33
 returning video number  34
 returning video number  35
 returning video nu